# Using the HyP3 SDK for Python
https://nbviewer.org/github/ASFHyP3/hyp3-sdk/blob/main/docs/sdk_example.ipynb#Submitting-Sentinel-1-InSAR-jobs

In [ ]:
import asf_search as asf
import hyp3_sdk as sdk
from hyp3_sdk import HyP3
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import cartopy
import geopandas as gpd
import pyproj
import antimeridian
import os
import zipfile
import rasterio
from utils import upload_file
import json
import ntpath 
import shutil
import yaml

## Helper functions

In [ ]:
def polygon_from_bounds(xmin, xmax, ymin, ymax):
    point1 = (float(xmin), float(ymax))
    point2 = (float(xmax), float(ymax))
    point3 = (float(xmax), float(ymin))
    point4 = (float(xmin), float(ymin))
    # Create a Shapely polygon from the points
    aoi = Polygon([point1, point2, point3, point4])
    return aoi

def transform_polygon(src_crs, dst_crs, geometry, always_xy=True):
    src_crs = pyproj.CRS(f"EPSG:{src_crs}")
    dst_crs = pyproj.CRS(f"EPSG:{dst_crs}") 
    transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=always_xy)
     # Transform the polygon's coordinates
    transformed_exterior = [
        transformer.transform(x, y) for x, y in geometry.exterior.coords
    ]
    # Create a new Shapely polygon with the transformed coordinates
    transformed_polygon = Polygon(transformed_exterior)
    return transformed_polygon

def plot_polygons(polygons, bounds, crs=4326, colour='dodgerblue', colours=[], proj='stereo'):
    plt.rcParams["figure.figsize"] = [10,8]
    # plot the the product geometries on a map
    east, west, south, north = bounds
    if proj=='stereo':
        ax = plt.axes(projection=cartopy.crs.SouthPolarStereo())
    else:
        ax = plt.axes(projection=cartopy.crs.PlateCarree())
    ax.set_extent((east, west, south, north), cartopy.crs.PlateCarree())
    ax.add_feature(cartopy.feature.LAND)
    ax.add_feature(cartopy.feature.OCEAN)
    #proj = cartopy.crs.SouthPolarStereo() if crs==3031 else cartopy.crs.PlateCarree()
    for i,polygon in enumerate(polygons):
        c = colours[i] if colours else colour
        ax.add_geometries(polygon, 
                        crs=cartopy.crs.PlateCarree(), 
                        alpha=0.3, 
                        edgecolor='black',
                        facecolor=c,
                        linewidth=1
                        #fillcolor='white'
                        ) 
    ax.gridlines(draw_labels=True)
    ax.add_feature(cartopy.feature.COASTLINE)
    plt.show()


## Authenticating to the API

The SDK will attempt to pull your [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) credentials out of `~/.netrc`
by default. Otherwise, you can add your credentials to the  `credentials/credentials_earthdata.yaml` file and the code below will read them in.

In [ ]:
with open("credentials/credentials_earthdata.yaml", "r", encoding='utf8') as f:
   earthdata_cfg = yaml.safe_load(f.read())
   uid = earthdata_cfg['login']
   pswd = earthdata_cfg['password']

In [ ]:
hyp3 = HyP3(username=uid, password=pswd)

# Data Search
- There are two main searching methods implemented in this notebook. If the scenes of interest are known, a list can be specified. If the scenes are not known, a search can be made using the asf.search function.

In [ ]:
search_list = True

## 1. Search with list

In [ ]:
if search_list:
    
    sar_software_comparison_20m_list = [
    
        # Antarctica - Law Dome (-66.163448 113.66396)
        'S1B_IW_SLC__1SSH_20211204T123606_20211204T123635_029875_039105_E7EC',
        'S1A_IW_SLC__1SSH_20211210T123647_20211210T123716_040946_04DCF0_9FBA',
        'S1B_IW_SLC__1SSH_20211216T123605_20211216T123634_030050_03968D_9501',
        # Antarctica - Ross Island (-77.7018, 166.7541)
        'S1A_IW_SLC__1SSH_20211208T124745_20211208T124815_040917_04DBED_5CFF',
        'S1A_IW_SLC__1SSH_20211220T124745_20211220T124815_041092_04E1C2_0475',
        'S1A_IW_SLC__1SSH_20220101T124744_20220101T124814_041267_04E7A2_1DAD',
        # Antarctica - Heard/McDonald Island (-53.1057, 73.543) 
        'S1A_IW_SLC__1SSH_20220103T141935_20220103T142005_041297_04E8B4_9B44',
        'S1A_IW_SLC__1SSH_20220108T142747_20220108T142815_041370_04EB2C_4E64',
        'S1A_IW_SLC__1SSH_20220115T141935_20220115T142005_041472_04EE7C_9E3D',
        # Antarctica - Antimeridian (-70, 180) approx - These Failed through hyp3
        'S1A_IW_SLC__1SSH_20220203T153614_20220203T153644_041750_04F7DF_8C3E',
        'S1A_IW_SLC__1SSH_20220122T153615_20220122T153645_041575_04F1EF_9254',
        # Antarctica - Antimeridian (-75, 180) approx
        'S1A_IW_SLC__1SDV_20191028T131928_20191028T131947_029659_0360CA_A1A0',
        'S1A_IW_SLC__1SDH_20161201T131103_20161201T131131_014186_016EB7_F766',
        # Australia - Queensland Forest (-25.533333 147.533333)
        'S1A_IW_SLC__1SDV_20220121T193807_20220121T193834_041563_04F181_5804',
        'S1A_IW_SLC__1SDV_20220202T084138_20220202T084206_041731_04F740_197D',
        'S1A_IW_SLC__1SDV_20220202T084204_20220202T084231_041731_04F740_400F',
    ] 

    sar_software_comparison_10m_list = [
        # Antarctica - Maitri Station (-70.76683, 11.7308)
        'S1B_IW_SLC__1SSH_20190327T195016_20190327T195045_015544_01D236_9504',
        'S1B_IW_SLC__1SSH_20190526T195018_20190526T195048_016419_01EE8D_53BC',
        'S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B',
        # Antarctica - Bharati Station (-69.4068, 76.19525)
        'S1B_IW_SLC__1SSH_20190223T222639_20190223T222706_015079_01C2E9_1D63',
        'S1A_IW_SLC__1SSH_20190605T222724_20190605T222751_027550_031BE1_AD3A',
        # Australia - Yaragadee (-29.0474, 115.3461) Descending (Only Avail)
        'S1B_IW_SLC__1SDV_20211201T214143_20211201T214210_029837_038FD8_81C3',
        'S1B_IW_SLC__1SDV_20211201T214208_20211201T214240_029837_038FD8_8B37',
        'S1B_IW_SLC__1SDV_20211213T214143_20211213T214210_030012_039555_8753',
        # Australia - SURAT QLD corner reflector (-27.0252,150.5759) - ASCENDING MODE (REQUIRED)
        'S1A_IW_SLC__1SSH_20220116T083314_20220116T083342_041483_04EED2_DB08',
        'S1A_IW_SLC__1SSH_20220104T083315_20220104T083342_041308_04E91B_8E5F',
        'S1B_IW_SLC__1SSH_20211216T123605_20211216T123634_030050_03968D_9501',
    ]
    
    iw_scene_list = []

    ew_scene_list = [
        'S1A_EW_GRDM_1SDH_20220225T174025_20220225T174129_042072_050315_7C35',
        'S1A_EW_GRDM_1SDH_20221023T174035_20221023T174139_045572_0572D2_CD12',
        'S1A_EW_GRDM_1SDH_20190924T112554_20190924T112659_029162_034FAB_41A7',
        'S1A_EW_GRDM_1SDH_20230118T112609_20230118T112713_046837_059DB3_9C85',
        'S1B_EW_GRDM_1SDH_20210221T234712_20210221T234816_025711_0310AD_3C16',
        'S1B_EW_GRDM_1SDH_20210304T000326_20210304T000430_025857_031578_789C',
    ]

    sm_scene_list = [
        'S1A_S3_SLC__1SSH_20230925T000119_20230925T000138_050476_06143F_154F',
        'S1A_S3_SLC__1SSH_20200108T000054_20200108T000113_030701_0384ED_8266'
    ]

    qld_CR_list = [
        # Australia - QLD corner reflector (-27.0252,150.5759)
        'S1A_IW_SLC__1SSH_20220104T083315_20220104T083342_041308_04E91B_8E5F',
        'S1A_IW_SLC__1SDV_20220111T192213_20220111T192240_041417_04ECBD_9F3B',
        'S1A_IW_SLC__1SSH_20220116T083314_20220116T083342_041483_04EED2_DB08',
        'S1A_IW_SLC__1SDV_20220123T192213_20220123T192240_041592_04F283_9D0E',
    ]

    qld_rainforest_timeseries_list = [
        'S1A_IW_SLC__1SDV_20221220T191402_20221220T191429_046419_058F94_70A0',
        'S1A_IW_SLC__1SDV_20221208T191403_20221208T191430_046244_058998_28A3',
        'S1A_IW_SLC__1SDV_20221126T191403_20221126T191430_046069_05839D_9DD9',
        'S1A_IW_SLC__1SDV_20221102T191403_20221102T191430_045719_0577C9_B76C',
        'S1A_IW_SLC__1SDV_20221009T191404_20221009T191431_045369_056CB2_5D9E',
        'S1A_IW_SLC__1SDV_20220915T191403_20220915T191430_045019_0560F2_B29C',
        'S1A_IW_SLC__1SDV_20220822T191402_20220822T191429_044669_05552A_4729',
        'S1A_IW_SLC__1SDV_20220729T191401_20220729T191428_044319_054A0A_E294',
        'S1A_IW_SLC__1SDV_20220705T191359_20220705T191426_043969_053FA4_EAFD',
        'S1A_IW_SLC__1SDV_20220611T191358_20220611T191425_043619_053532_39C3',
        'S1A_IW_SLC__1SDV_20220424T191355_20220424T191422_042919_051FA1_3965',
        'S1A_IW_SLC__1SDV_20220331T191354_20220331T191421_042569_0513ED_EDBB',
        'S1A_IW_SLC__1SDV_20220307T191354_20220307T191421_042219_050811_3A95',
        'S1A_IW_SLC__1SDV_20220211T191354_20220211T191421_041869_04FC03_7559',
        'S1A_IW_SLC__1SDV_20220130T191354_20220130T191421_041694_04F5F9_1AFD',
        'S1A_IW_SLC__1SDV_20220106T191355_20220106T191422_041344_04EA48_E185',
    ]


    # search the scene list with the specified product type
    iw_results = asf.granule_search(iw_scene_list, asf.ASFSearchOptions(processingLevel='SLC'))
    ew_results = asf.granule_search(ew_scene_list, asf.ASFSearchOptions(processingLevel='GRD_MD'))
    sm_results = asf.granule_search(sm_scene_list, asf.ASFSearchOptions(processingLevel='SLC'))
    qld_cr_results = asf.granule_search(qld_CR_list, asf.ASFSearchOptions(processingLevel='SLC'))
    qld_rainforest_results = asf.granule_search(qld_rainforest_timeseries_list, asf.ASFSearchOptions(processingLevel='SLC'))
    sar_software_comparison_20m_results = asf.granule_search(sar_software_comparison_20m_list, asf.ASFSearchOptions(processingLevel='SLC'))
    sar_software_comparison_10m_results = asf.granule_search(sar_software_comparison_10m_list, asf.ASFSearchOptions(processingLevel='SLC'))

## 2. Search for scenes using asf.search

In [ ]:
# searh through time
if not search_list:
    # prod = 'GRD_HS' # IW
    # prod = 'GRD_MD' # EW

    wkt = "POINT (147.533333 -25.533333)" # Austraia - Queensland Forest Timeseries
    wkt = "POINT (152.604 -26.637)" # Austraia - Queensland Rainforest Forest

    # prod = 'GRD_HD'
    # prod = 'GRD_MS'
    # prod = 'GRD_FD'
    prod = 'SLC'

    # acquisition mode
    mode = 'IW'
    #mode = 'EW'
    #mode = ['S1','S2','S3','S4','S5','S6']

    # start_date = '2021-12-01T00:00:00Z'
    # end_date = '2022-02-28T00:00:00Z'

    start_date = '2022-01-01T00:00:00Z'
    end_date = '2022-12-31T00:00:00Z'

    results = asf.search(platform=[asf.PLATFORM.SENTINEL1], 
                        intersectsWith=wkt, 
                        maxResults=1000, 
                        processingLevel=prod,
                        beamMode=mode,
                        start=start_date,
                        end=end_date,
                        #frame=1097
                        )
    search_scenes = []
    for r in reversed(results):
        id_ = r.__dict__['umm']['GranuleUR'].split('-')[0]
        search_scenes.append(id_)
    

In [ ]:
results = sar_software_comparison_20m_results
print(len(results))
# for r in results[0:1]:
#      for i,x in enumerate(r.__dict__['umm']['AdditionalAttributes']):
#           print(i,x)

In [ ]:
# The code below will pull the location information for each scene
# This data can be saved to json file, or is plotted below
scene_locations = {}
scene_locations_3031 = {}
for r in reversed(results):
    id = r.__dict__['umm']['GranuleUR'].split('-')[0]
    print(r.__dict__['umm']['GranuleUR'].split('-')[0])
    print(r.__dict__['umm']['AdditionalAttributes'][1]['Values'],
          r.__dict__['umm']['AdditionalAttributes'][17],
          r.__dict__['umm']['AdditionalAttributes'][18],
          r.__dict__['umm']['AdditionalAttributes'][29],
          r.__dict__['umm']['OrbitCalculatedSpatialDomains'][0]) #[1] #['FRAME_NUMBER']
    print(r.__dict__['umm']['SpatialExtent'])
    points = (r.__dict__['umm']['SpatialExtent']['HorizontalSpatialDomain']
              ['Geometry']['GPolygons'][0]['Boundary']['Points'])
    points = [(p['Longitude'],p['Latitude']) for p in points]
    poly = Polygon(points)
    poly_3031 = transform_polygon(4326, 3031, poly, always_xy=True)
    poly = antimeridian.fix_polygon(poly)
    scene_locations[id] = poly
    scene_locations_3031[id] = poly_3031

# save locations to file
save = False
if save: 
    df = pd.DataFrame.from_dict(scene_locations, orient='index')
    df = df.rename(columns = {0:'geometry'})
    gdf = gpd.GeoDataFrame(df, crs=4326, geometry='geometry')
    gdf.to_file('qld_timeseries_scene_locations.json')

#plot
colours = ['green' if 'IW' in x else ('blue' if 'EW' in x else 'red') for x in scene_locations.keys()]
plot_polygons(scene_locations.values(), (-180,180,-90,-53), colours=colours)
plot_polygons(scene_locations.values(), (110,160,-45,-8), colours=colours, proj='plate')

In [ ]:
# Downlod the scenes if required
download = False
download_folder = 'data'
if download:
    session = asf.ASFSession()
    session.auth_with_creds(uid,pswd)
    # download all results
    scene_paths = []
    for scene in results:
        name = scene.__dict__['umm']['GranuleUR'].split('-')[0]
        print(name)
        path = os.path.join(download_folder, name)
        scene.download(path=download_folder, session=session)
        scene_paths.append(path)


## Submitting jobs

The SDK provides a submit method for [all supported job types](https://hyp3-docs.asf.alaska.edu/products/).

### Submitting Sentinel-1 RTC jobs

Sentinel-1 Radiometric Terrain Correction (RTC) jobs are submitted using [ESA granule IDs](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/naming-conventions).
The example granules below can be viewed  in [ASF Search here](https://search.asf.alaska.edu/#/?zoom=7.08772014623877&center=-141.733866,58.498008&resultsLoaded=true&granule=S1A_IW_SLC__1SDV_20210214T154835_20210214T154901_036588_044C54_8494-SLC&searchType=List%20Search&searchList=S1A_IW_SLC__1SDV_20210214T154835_20210214T154901_036588_044C54_8494-SLC,S1B_IW_SLC__1SDV_20210210T153131_20210210T153159_025546_030B48_B568-SLC,S1A_IW_SLC__1SDV_20210210T025526_20210210T025553_036522_0449E2_7769-SLC,S1A_IW_SLC__1SDV_20210210T025501_20210210T025528_036522_0449E2_3917-SLC,S1B_IW_SLC__1SDV_20210209T030255_20210209T030323_025524_030A8D_7E88-SLC,S1B_IW_SLC__1SDV_20210209T030227_20210209T030257_025524_030A8D_5BAF-SLC,S1A_IW_SLC__1SDV_20210202T154835_20210202T154902_036413_044634_01A1-SLC).

In [ ]:
projects = {
    'sar-software-comparison-20m': sar_software_comparison_20m_list,
    'qld-cr-2022-01': qld_CR_list,
    #'qld-timeseries-10m-0122-0622' : search_scenes,
    'qld-rainforest-timeseries-20m-2022' : qld_rainforest_timeseries_list,
}


In [ ]:
# set the proj name
proj_name = 'sar-software-comparison-20m'
#proj_name = 'qld-cr-2022-01'

In [ ]:
# submit jobs
rtc_jobs = sdk.Batch()
scene_list = projects[proj_name]
for g in scene_list:
    rtc_jobs += hyp3.submit_rtc_job(g, 
                                    include_dem=True, #include dem in final product
                                    include_inc_map=True,  #include dem map in final product
                                    include_rgb=True,  #include rgb img in final product
                                    include_scattering_area=True, #include scat area in final product
                                    name=proj_name,
                                    resolution=20,
                                    dem_name='copernicus')
print(rtc_jobs)


Here we've given each job the name `rtc-example`, which we can use later to search for these jobs.

`HyP3.submit_rtc_job` also accepts
[keyword arguments](https://hyp3-docs.asf.alaska.edu/using/sdk_api/#hyp3_sdk.hyp3.HyP3.submit_rtc_job)
to customize the RTC products to your application.

### Watch Jobs

In [ ]:
rtc_jobs = hyp3.find_jobs(name=proj_name)
rtc_jobs = hyp3.watch(rtc_jobs)

### View which jobs have been completed

In [ ]:
rtc_jobs = hyp3.find_jobs(name=proj_name)
rtc_jobs = hyp3.refresh(rtc_jobs)
running_jobs = rtc_jobs.filter_jobs(succeeded=False, running=True, failed=False)
print(f'Number of running jobs: {len(running_jobs)}')
succeeded_jobs = rtc_jobs.filter_jobs(succeeded=True, running=False, failed=False)
print(f'Number of succeeded jobs: {len(succeeded_jobs)}')
failed_jobs = rtc_jobs.filter_jobs(succeeded=False, running=False, failed=True)
print(f'Number of failed jobs: {len(failed_jobs)}')

### Download, Unzip and Push to S3
- The following blocks of code will download the scene from the asf to the local machine. The scene will then be unzipped and the files uploaded to the specified s3 bucket.

In [ ]:
download_folder = '/data/hyp3-gamma'
s3_bucket = 'deant-data-public-dev'
s3_bucket_folder = proj_name
push_to_s3 = True
delete_after_upload = True

In [ ]:
# set credentials for pushing to s3 bucket
with open('credentials/credentials_aws.yaml', "r", encoding='utf8') as f:
    aws_cfg = yaml.safe_load(f.read())
    # set all keys as environment variables
    for k in aws_cfg.keys():
        os.environ[k] = aws_cfg[k]

In [ ]:
def make_timing_dict():
    # timing file with same structure as used for rtc-opera and
    # pyrosar. Fill with zeros.
    return {
        "Download Scene":	0,
        "Download Orbits":	0,
        "Download DEM":	0,
        "RTC Processing":	0,
        "S3 Upload":	0,
        "Delete Files":	0,
        "Total":	0,
    }

In [ ]:
for i,job in enumerate(succeeded_jobs):
    print(f'scene {i+1} of {len(succeeded_jobs)}')
    # download
    file_list = job.download_files(download_folder)
    process_time = job.processing_times
    _, filename = ntpath.split(str(file_list[0]))
    st = filename.split('_')[2]
    scene = [x for x in projects[proj_name] if st in x][0]
    print(scene)
    # unzip
    with zipfile.ZipFile(file_list[0], 'r') as zip_ref:
        zip_ref.extractall(download_folder)
    os.remove(file_list[0])
    prod_folder = os.path.join(download_folder, str(file_list[0]).replace('.zip',''))
    tif_file = [x for x in os.listdir(prod_folder) 
                    if (x.endswith('.tif') and any(ph in x for ph in ['HH','HV','VV','VH']))][0]
    # get the crs from the tif
    with rasterio.open(str(os.path.join(prod_folder, tif_file))) as src:
        crs = src.meta['crs']
        crs = str(crs).split(':')[-1]
    #make the timing file
    timing_dict = make_timing_dict()
    timing_dict['Total'] = process_time
    timing_dict['RTC Processing'] = process_time
    timing_path = os.path.join(prod_folder,f'{scene}_timing.json')
    with open(timing_path, 'w') as fp:
        json.dump(timing_dict, fp)
    for f in os.listdir(prod_folder):
        f_path = os.path.join(prod_folder, f)
        if push_to_s3:
            bucket_folder = f"{s3_bucket_folder}/hyp3-gamma/glo_30/{crs}/{scene}"
            object_name = f"{bucket_folder}/{f}"
            upload_file(f_path,s3_bucket,object_name)
    if delete_after_upload:
        shutil.rmtree(prod_folder)

### Submitting Sentinel-1 InSAR jobs

The SDK can also submit Sentinel-1 Interferometric Synthetic Aperture Radar (InSAR) jobs which processes
reference and secondary granule pairs.

For a particular reference granule, we may want to use the nearest and next-nearest temporal neighbor granules as secondary
scenes. To programmatically find our secondary granules for a reference granule, We'll define a `get_nearest_neighbors`
function that uses the [baseline stack](https://docs.asf.alaska.edu/asf_search/ASFProduct/#stack) method from `asf_search`:

In [ ]:
from typing import Optional

def get_nearest_neighbors(granule: str, max_neighbors: Optional[int] = None) -> asf.ASFSearchResults:
    #granule = asf.granule_search(granule)[-1] #original code commented out because didnt work...
    granule = asf.granule_search(granule)[0]
    stack = reversed([item for item in granule.stack() if item.properties['temporalBaseline'] < 0])
    results = asf.ASFSearchResults(stack)[:max_neighbors]
    return results

Now, using the example granule list for our RTC jobs as the reference scenes, we can find their nearest and next-nearest neighbor granules, and submit them
as pairs for InSAR processing.

In [ ]:
from tqdm.auto import tqdm  # For a nice progress bar: https://github.com/tqdm/tqdm#ipython-jupyter-integration

granules = [
    'S1A_IW_SLC__1SDV_20210214T154835_20210214T154901_036588_044C54_8494',
    'S1B_IW_SLC__1SDV_20210210T153131_20210210T153159_025546_030B48_B568',
    'S1A_IW_SLC__1SDV_20210210T025526_20210210T025553_036522_0449E2_7769',
    'S1A_IW_SLC__1SDV_20210210T025501_20210210T025528_036522_0449E2_3917',
    'S1B_IW_SLC__1SDV_20210209T030255_20210209T030323_025524_030A8D_7E88',
    'S1B_IW_SLC__1SDV_20210209T030227_20210209T030257_025524_030A8D_5BAF',
    'S1A_IW_SLC__1SDV_20210202T154835_20210202T154902_036413_044634_01A1',
]

insar_jobs = sdk.Batch()
for reference in tqdm(granules):
    neighbors = get_nearest_neighbors(reference, max_neighbors=2)
    for secondary in neighbors:
        print(f'Reference: {reference}')
        print(f'Secondary: {secondary.properties["sceneName"]}')
        insar_jobs += hyp3.submit_insar_job(
            reference, 
            secondary.properties['sceneName'], 
            name='insar-example-extra',
            include_look_vectors = True,
            include_los_displacement = True,
            include_inc_map = True,
            looks = '20x4', #'20x4', '10x2'
            include_dem = True,
            include_wrapped_phase = True,
            apply_water_mask = True,
            include_displacement_maps = True
            )
print(insar_jobs)

### Watch Jobs

In [ ]:
insar_jobs = hyp3.find_jobs(name='insar-example-extra')
insar_jobs = hyp3.watch(insar_jobs)

In [ ]:
insar_jobs = hyp3.refresh(insar_jobs)
succeeded_jobs = insar_jobs.filter_jobs(succeeded=True, running=False, failed=False)
print(f'Number of succeeded jobs: {len(succeeded_jobs)}')
failed_jobs = insar_jobs.filter_jobs(succeeded=False, running=False, failed=True)
print(f'Number of failed jobs: {len(failed_jobs)}')

### download

In [ ]:
file_list = succeeded_jobs.download_files()
file_list

## Check status

Like RTC jobs, `HyP3.submit_insar_job` accepts
[keyword arguments](https://hyp3-docs.asf.alaska.edu/using/sdk_api/#hyp3_sdk.hyp3.HyP3.submit_insar_job)
to customize the InSAR products to your application.

### Submitting autoRIFT jobs

AutoRIFT supports processing Sentinel-1, Sentinel-2, or Landsat-8 Collection 2 pairs.
* Sentinel-1 jobs are submitted using [ESA granule IDs](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-1-sar/naming-conventions)
* Sentinel-2 jobs are submitted using [ESA granule IDs](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/naming-convention)
* Landsat-8 Collection 2 jobs are submitted using [USGS scene IDs](https://www.usgs.gov/faqs/what-naming-convention-landsat-collection-2-level-1-and-level-2-scenes?qt-news_science_products=0#qt-news_science_products)

In [ ]:
autorift_pairs = [
    # Sentinel-1 ESA granule IDs
    ('S1A_IW_SLC__1SSH_20170221T204710_20170221T204737_015387_0193F6_AB07',
     'S1B_IW_SLC__1SSH_20170227T204628_20170227T204655_004491_007D11_6654'),
    # Sentinel-2 ESA granule IDs
    ('S2B_MSIL1C_20200612T150759_N0209_R025_T22WEB_20200612T184700',
     'S2A_MSIL1C_20200627T150921_N0209_R025_T22WEB_20200627T170912'),
    # Landsat 8
    ('LC08_L1TP_009011_20200703_20200913_02_T1',
     'LC08_L1TP_009011_20200820_20200905_02_T1'),
]

autorift_jobs = sdk.Batch()
for reference, secondary in autorift_pairs:
    autorift_jobs += hyp3.submit_autorift_job(reference, secondary, name='autorift-example')
print(autorift_jobs)

In [ ]:
autorift_jobs = hyp3.watch(autorift_jobs)

In [ ]:
autorift_jobs = hyp3.refresh(autorift_jobs)
succeeded_jobs = autorift_jobs.filter_jobs(succeeded=True, running=False, failed=False)
print(f'Number of succeeded jobs: {len(succeeded_jobs)}')
failed_jobs = autorift_jobs.filter_jobs(succeeded=False, running=False, failed=True)
print(f'Number of failed jobs: {len(failed_jobs)}')

In [ ]:
file_list = succeeded_jobs.download_files()
file_list